In [2]:
import os
import cv2
import wandb

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa

from os import path
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, ZeroPadding2D
from tensorflow.keras.layers import Activation
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam, RMSprop
from sklearn.utils import shuffle
from tensorflow.keras.callbacks import LambdaCallback, EarlyStopping, ReduceLROnPlateau

print(tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None))

wandb.login()

config = {"lr": 1e-3,
          "num_epoch": 50,
          "size": 256,
          "optimizer": "adam",
          "net": "simple",
          "reduceLRO": 0.3,
          "batch_size": 16}


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
True


wandb: Currently logged in as: camperko (use `wandb login --relogin` to force relogin)


True

In [ ]:
def resize_image(in_image, image_name, is_content):
    resized_image = cv2.resize(in_image, (config["size"], config["size"]), interpolation = cv2.INTER_AREA)
    cv2.imwrite('data/resized/' + 'content' if is_content else 'style' + image_name, resized_image)
    return resized_image


In [ ]:
# Load style data


In [ ]:
# Load content data